# MLP para Predição de Indicadores Sociais

### Importação das bibliotecas

In [ ]:
import pandas as pd
import numpy as pd
import xlrd
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import tqdm

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix


### Extração dos Arquivos

In [ ]:

arquivo = xlrd.open_workbook('datasets/Tabelas1a21.xls')

planilhas_alvo = ['tab2', 'tab4', 'tab6', 'tab8', 'tab19']
abas = {}

for nome in planilhas_alvo:
    abas[nome] = arquivo.sheet_by_name(nome)
    
capitais = [
    'Porto Velho', 'Rio Branco', 'Manaus', 'Boa Vista', 'Belém', 'Macapá', 'Palmas',
    'São Luís', 'Teresina', 'Fortaleza', 'Natal', 'João Pessoa', 'Recife', 'Maceió', 
    'Aracaju', 'Salvador', 'Belo Horizonte', 'Vitória', 'Rio de Janeiro', 'São Paulo',
    'Curitiba', 'Florianópolis', 'Porto Alegre', 'Campo Grande', 'Cuiabá', 'Goiânia', 'Brasília'
]

capitais_treinamento = capitais[:20]
capitais_teste = capitais[20:]

print(f"Capitais para treino ({len(capitais_treinamento)}): {capitais_treinamento}")
print(f"Capitais para teste ({len(capitais_teste)}): {capitais_teste}")


Capitais para treino (20): ['Porto Velho', 'Rio Branco', 'Manaus', 'Boa Vista', 'Belém', 'Macapá', 'Palmas', 'São Luís', 'Teresina', 'Fortaleza', 'Natal', 'João Pessoa', 'Recife', 'Maceió', 'Aracaju', 'Salvador', 'Belo Horizonte', 'Vitória', 'Rio de Janeiro', 'São Paulo']
Capitais para teste (7): ['Curitiba', 'Florianópolis', 'Porto Alegre', 'Campo Grande', 'Cuiabá', 'Goiânia', 'Brasília']


Tabela 2 - Domícilios por Sexo

In [16]:
def extrair_tabela2():
    aba2 = abas['tab2']
    dados = {}
    
    # achar ano 2000
    start_linha = None
    for i in range(aba2.nrows):
        if '2000' in str(aba2.cell_value(i, 0)):
            start_linha = i + 1
            break
    
    if start_linha is None:
        raise ValueError("Não encontrou dados de 2000 na Tabela 2")
    
    for i in range(start_linha, aba2.nrows):
        capital = aba2.cell_value(i, 0).strip()
        if capital in capitais_treinamento:
            perc_mulheres = aba2.cell_value(i, 5)
            dados[capital] = perc_mulheres
    
    return dados


Tabela 4 (Distribuição etária - 2000)

In [ ]:
def extrair_tabela4():
    aba4 = abas['tab4']
    dados = {}

    start_linha = None
    for i in range(aba4.nrows):
        if '2000' in str(aba4.cell_value(i, 0)):
            start_linha = i + 1
            break
    
    if start_linha is None:
        raise ValueError("Não encontrou dados de 2000 na Tabela 4")
    
    for i in range(start_linha, aba4.nrows):
        capital = aba4.cell_value(i, 0).strip()
        if capital in capitais_treinamento:
            perc_25_29 = aba4.cell_value(i, 4)  # 25 a29 
            perc_35_39 = aba4.cell_value(i, 6)  # 35 a 39  
            perc_45_49 = aba4.cell_value(i, 8)  # 45 a 49
            dados[capital] = {
                'perc_25_29': perc_25_29,
                'perc_35_39': perc_35_39,
                'perc_45_49': perc_45_49
            }
    
    return dados